In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.

In [3]:
from huggingface_hub import interpreter_login
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import __version__, BartTokenizer, BartForConditionalGeneration
import evaluate
import wandb
from torch import cuda
import time

In [5]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: amaanrzv786 (amaanrzv786-research) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [6]:
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) n


In [7]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [24]:
wandb.init(project="BART_summarization")

config = wandb.config
config.TRAIN_BATCH_SIZE = 4
config.VALID_BATCH_SIZE = 4
config.TRAIN_EPOCHS = 15
config.LEARNING_RATE = 1e-4
config.SEED = 42
config.MAX_LEN = 512
config.SUMMARY_LEN = 150

torch.manual_seed(config.SEED)
np.random.seed(config.SEED)
torch.backends.cudnn.deterministic = True

new_repo = "Amaan39/text-summarizer"
repo_name = "EducativeCS2023/bart-base-summarization"

In [15]:
df = pd.read_csv('./BBCarticles.csv' ,encoding='latin-1')
df = df[['Text','Summary']]
df.Text = 'summarize: ' + df.Text

split = 0.5
train_dataset=df.sample(frac=split,random_state = config.SEED)
eval_dataset=df.drop(train_dataset.index).sample(frac=split,random_state = config.SEED).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("train_dataset", train_dataset.shape)
print("eval_dataset", eval_dataset.shape)

df.head(3)

train_dataset (1112, 2)
eval_dataset (556, 2)


,Text,Summary
0,summarize: Ad sales boost Time Warner profit\n...,TimeWarner said fourth quarter sales rose 2% t...
1,summarize: Dollar gains on Greenspan speech\n\...,The dollar has hit its highest level against t...
2,summarize: Yukos unit buyer faces loan claim\n...,Yukos' owner Menatep Group says it will ask Ro...


In [16]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.Summary = self.data.Summary
        self.Text = self.data.Text

    def __len__(self):
        return len(self.Summary)

    def __getitem__(self, index):
        Text = str(self.Text[index])
        Text = ' '.join(Text.split())

        Summary = str(self.Summary[index])
        Summary = ' '.join(Summary.split())

        source = self.tokenizer([Text], max_length= self.source_len, padding='max_length', truncation=True, return_tensors='pt')
        target = self.tokenizer([Summary], max_length= self.summ_len, padding='max_length', truncation=True, return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long),
            'source_mask': source_mask.to(dtype=torch.long),
            'target_ids': target_ids.to(dtype=torch.long)
        }

In [20]:
tokenizer = BartTokenizer.from_pretrained(repo_name)

tokenizer.push_to_hub(new_repo)

training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
eval_set = CustomDataset(eval_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

training_loader = DataLoader(training_set, batch_size=config.TRAIN_BATCH_SIZE, shuffle=True, num_workers=0)
eval_loader = DataLoader(eval_set, batch_size=config.VALID_BATCH_SIZE, shuffle=False, num_workers=0)

In [21]:
model = BartForConditionalGeneration.from_pretrained(repo_name)
model = model.to(device)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

wandb.watch(model, log="all")

config.json:   0%|          | 0.00/1.75k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/558M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

In [22]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for i,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        labels = y[:, 1:].clone().detach()
        labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=labels)
        loss = outputs[0]
        if i%10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if i%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [25]:
for epoch in range(config.TRAIN_EPOCHS):
    train(epoch, tokenizer, model, device, training_loader, optimizer)

model.push_to_hub(new_repo)

Epoch: 0, Loss:  0.08625998347997665
Epoch: 1, Loss:  0.12231718003749847
Epoch: 2, Loss:  0.0729299783706665
Epoch: 3, Loss:  0.03903847932815552
Epoch: 4, Loss:  0.10155278444290161
Epoch: 5, Loss:  0.08357670903205872
Epoch: 6, Loss:  0.08549070358276367
Epoch: 7, Loss:  0.0635322853922844
Epoch: 8, Loss:  0.0569961853325367
Epoch: 9, Loss:  0.06257050484418869
Epoch: 10, Loss:  0.055785708129405975
Epoch: 11, Loss:  0.019372988492250443
Epoch: 12, Loss:  0.074137382209301
Epoch: 13, Loss:  0.014935025945305824
Epoch: 14, Loss:  0.046601586043834686


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Amaan39/text-summarizer/commit/b0109b3decf76df22be51058ae0b10eb261da537', commit_message='Upload BartForConditionalGeneration', commit_description='', oid='b0109b3decf76df22be51058ae0b10eb261da537', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Amaan39/text-summarizer', endpoint='https://huggingface.co', repo_type='model', repo_id='Amaan39/text-summarizer'), pr_revision=None, pr_num=None)

In [26]:
def predict(tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for i, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask,
                max_length=150,
                num_beams=2,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if i%100==0:
                print(f'Completed {i}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [27]:
pd.set_option('display.max_colwidth', None)

tik = time.time()
predictions, actuals = predict(tokenizer, model, device, eval_loader)
results = pd.DataFrame({'predictions':predictions,'actuals':actuals})
results.to_csv('predictions.csv')
tok = time.time()
print("time taken", tok-tik, "seconds")
results.head()

Completed 0
Completed 100
time taken 224.54710984230042 seconds


,predictions,actuals
0,"Rusedski said: ""I tried to order some but didn't receive any and I haven't got any yet.""The first chance we will get to do that is at the players meeting on the Saturday before the Australian Open.""""You would think they would have been available in December as it can take two months for the body to respond.""The good thing is that there is now a 100% guarantee, so hopefully all this will never happen again,"" said Rusedsk.""I would have loved to have had these things available as soon as possible but it's a lot of work to make sure they have gone through rigorous testing.""This event comes in the hottest period of the year, so you would hope the stuff would be available for","""You would think they would have been available in December as it can take two months for the body to respond.""The first chance we will get to do that is at the players meeting on the Saturday before the Australian Open.""""The good thing is that there is now a 100% guarantee, so hopefully all this will never happen again,"" said Rusedski.Greg Rusedski has criticised the governing body of men's tennis for not releasing contamination-free supplements in time for the new season.""This event comes in the hottest period of the year, so you would hope the stuff would be available for it.""""Hopefully after the Australian Open we won't have to discuss this any more."""
1,"pey said the UK housing market had proved ""challenging"" last year.Overall, B&B said that ""housing market fundamentals remain strong"".The UK property market remains robust despite the recent slowdown, according to mortgage lender Bradford & Bingley and housebuilder George Wimpey.Wimpey reported a 19% rise in profits to Â£450.7m and said recent new home reservations were better than expected.But B&R - which has a 22% share of the UK buy-to-let mortgage market - said that while rates of growth were moderating, the sector ""continues to grow at a rate considerably above that of the whole mortgage market"".Shares in WimpeY were up 6","Wimpey said the UK housing market had proved ""challenging"" last year.But B&B - which has a 22% share of the UK buy-to-let mortgage market - said that while rates of growth were moderating, the sector ""continues to grow at a rate considerably above that of the whole mortgage market"".B&B said the buy-to-let market - in which the bank is a major player - would continue to grow much faster than the wider mortgage market.Recent housing market surveys have indicated that the UK property market has cooled in recent months after several years of rapid growth.Overall, B&B said that ""housing market fundamentals remain strong"".The UK property market remains robust despite the"
2,"M has received 25,772 US patents in that period and reportedly has more than 40,000 current patents.Computer giant IBM says 500 of its software patents will be released into the open development community.IBM described the step as a ""new era"" in how it dealt with intellectual property and promised further patents would be made freely available.Mr Jollans likened the plan for a patent commons to the way the internet was developed and said everyone could take advantage of the result of collaboration.In a statement, Dr John E. Kelly, IBM senior vice president, Technology and Intellectual Property, said: ""true innovation leadership is about more than just the numbers of patents granted.The company said it wanted to encourage other firms to release patents","For each of the past 12 years IBM has been granted more US patents than any other company.The company said it wanted to encourage other firms to release patents into what it called a ""patent commons"".Mr Jollans likened the plan for a patent commons to the way the internet was developed and said everyone could take advantage of the result of collaboration.IBM was granted 3,248 patents in 2004, more than any other firm in the US, the New York Times reports.IBM has received 25,772 US patents in that period and re

In [29]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=b33d08ad9469be2b44c035cabf10b1553531b88b19ccff2ec0b113e2d863d9d6
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [30]:
rouge_score = evaluate.load("rouge")

scores = rouge_score.compute(
    predictions=results['predictions'], references=results['actuals']
)
pd.DataFrame([scores]).T.head()

,0
rouge1,0.708124
rouge2,0.612698
rougeL,0.540386
rougeLsum,0.540635
